In [1]:
import pandas as pd
import sqlite3

In [2]:
conn=sqlite3.connect("E:\pythonfiles\Db-IMDB-Assignment.db")

Question 1 :
List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [3]:
query="""
Select p.Name as Director, m.title as Movie, SUBSTRING(m.year,-4,4) as LeapYear, ge.name from 
M_Director d join Person P
on d.PID=p.PID
join M_Genre g
on d.MID=g.MID
join Genre ge
on g.GID=ge.GID
join Movie m
on g.MID=m.MID
where ge.Name like "%Comedy%" 
and (LeapYear%4=0 and not LeapYear%100=0) or LeapYear%400=0 
;
"""
ques= pd.read_sql_query(query,conn)
display(ques)
#https://www.w3schools.com/sql/func_mysql_substring.asp(for [reprocessing of year column])

,Director,Movie,LeapYear,Name
0,Milap Zaveri,Mastizaade,2016,Comedy
1,Danny Leiner,Harold & Kumar Go to White Castle,2004,"Adventure, Comedy"
2,Anurag Kashyap,Gangs of Wasseypur,2012,"Action, Comedy, Crime"
3,Frank Coraci,Around the World in 80 Days,2004,"Action, Adventure, Comedy"
4,Griffin Dunne,The Accidental Husband,2008,"Comedy, Romance"
...,...,...,...,...
275,Siddharth Anand Kumar,Let's Enjoy,2004,Comedy
276,Amma Rajasekhar,Sathyam,2008,"Action, Comedy"
277,Oliver Paulus,Tandoori Love,2008,"Comedy, Drama, Romance"
278,Raja Chanda,Le Halua Le,2012,"Comedy, Drama, Romance"


Question 2 :
List the names of all the actors who played in the movie 'Anand' (1971) 

In [4]:
#SELECT Query

query2="""
Select Name from Person Where PID IN 
(select TRIM(PID) from M_Cast Where MID IN
(Select MID from Movie Where title='Anand')
);
"""
ques2= pd.read_sql_query(query2,conn)
display(ques2)

#OR


#JOIN query

querya="""
Select Name, TRIM(c.PID) as PersonID from Person p
JOIN M_Cast c
ON p.PID=PersonID
JOIN Movie m
ON c.MID=m.MID
where m.title='Anand'
;"""
quesa=pd.read_sql_query(querya,conn)
quesa


,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


,Name,PersonID
0,Amitabh Bachchan,nm0000821
1,Rajesh Khanna,nm0004435
2,Brahm Bhardwaj,nm0080231
3,Ramesh Deo,nm0219946
4,Seema Deo,nm0219947
5,Dev Kishan,nm0222743
6,Durga Khote,nm0451862
7,Lalita Kumari,nm0474931
8,Lalita Pawar,nm0667985
9,Atam Prakash,nm0695155


Question 3 :
List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 
and > 1990) .

In [5]:
query3="""
 Select Name from Person Where PID in
(Select TRIM(PID) from M_Cast Where MID in
(Select TRIM(MID) from Movie Where year<1970)
Intersect
Select TRIM(PID) from M_Cast Where MID in
(Select TRIM(MID) from Movie Where year>1990)
);
"""
ques3= pd.read_sql_query(query3,conn)
ques3

,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
...,...
328,Poonam
329,Jamila Massey
330,K.R. Vijaya
331,Sethi


In [6]:
query4="""
Select p.Name as Directors_name, Count(d.PID) as Movie_Count From
Person p Join M_Director d
On p.PID=d.PID
Group by d.PID
Having Count(d.PID)>10
order by Count(d.PID) desc
limit 10;
"""
ques4= pd.read_sql_query(query4,conn)
ques4

,Directors_name,Movie_Count
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Shakti Samanta,19
8,Basu Chatterjee,19
9,Subhash Ghai,18


Question 5 : 
A.For each year, count the number of movies in that year that had only female actors. 

B. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [14]:
query5A="""
Select SUBSTRING(year,-4,4) as year, Count(MID) as Number_Of_Female_Only_Movies from Movie WHERE MID NOT IN
(Select c.MID from M_Cast c JOIN Person P
ON TRIM(c.PID)=TRIM(p.PID)
Where p.GENDER='Male')
Group by year
Order by year
;

"""
ques5A= pd.read_sql_query(query5A,conn)
ques5A

,year,Number_Of_Female_Only_Movies
0,1939,1
1,1999,1
2,2000,1
3,2018,1


In [14]:
query5B = """SELECT m.year as Year, COUNT(m.MID) as Total_Movie_Count,female_mcount, (female_mcount * 100.0 / COUNT(m.MID)) as Percentage FROM Movie m JOIN
            (SELECT COUNT(TRIM(MID)) as female_mcount, m.year as Year FROM Movie m WHERE TRIM(MID) NOT IN
            (SELECT DISTINCT  TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
            (SELECT p.PID FROM Person p  WHERE TRIM(p.Gender) = 'Male'))  GROUP BY m.year)as fmovies_count 
            ON  m.year = fmovies_count.year 
            GROUP BY m.year
 
"""
ques5B = pd.read_sql_query(query5B,conn)
ques5B

,Year,Total_Movie_Count,female_mcount,Percentage
0,1939,2,1,50.000000
1,1999,66,1,1.515152
2,2000,64,1,1.562500
3,I 2018,10,1,10.000000


Question 6 :
Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [7]:
query6="""
Select Title, MAX(NumberOfPeople) as Castcount from
(Select Title, count(Distinct(c.pid)) as NumberOfPeople
from Movie m Join M_Cast c
ON m.MID=c.MID
Group by m.MID
Order by NumberOfPeople desc) as X;
"""
ques6= pd.read_sql_query(query6,conn)
ques6

,Title,Castcount
0,Ocean's Eight,238


Question 7 :
A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [8]:
query7  = """
WITH 
UNIQUE_YEAR AS (SELECT DISTINCT "year" FROM Movie)

SELECT D."year" AS START, D."year"+9 AS END, COUNT(*) AS 'COUNT' 
FROM UNIQUE_YEAR D
JOIN Movie M on M."year" >= START AND M."year"<= END
GROUP BY END 
ORDER BY COUNT(*) DESC 
LIMIT 1;
"""
ques7 = pd.read_sql_query(query7,conn)
ques7

,START,END,COUNT
0,2008,2017,1126


Question 8:
Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [10]:
query8 = """ 
WITH Movie_Year AS (
SELECT DISTINCT TRIM(MC.PID) AS PID, TRIM(M.Year) AS YEAR, 
ROW_NUMBER() OVER (PARTITION BY TRIM(MC.PID) ORDER BY Year) Row_Num 
FROM Movie M JOIN M_Cast MC ON TRIM(M.MID)=TRIM(MC.MID) 
)

SELECT DISTINCT Name FROM Person WHERE PID NOT IN (
SELECT DISTINCT M1.PID FROM Movie_Year M1 JOIN Movie_Year M2 
ON M1.PID=M2.PID AND M1.Row_Num+1=M2.Row_Num
WHERE M2.Year-M1.Year >= 3  ) """
ques8 = pd.read_sql_query(query8,conn)
ques8

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
...,...
31090,Kamika Verma
31091,Dhorairaj Bhagavan
31092,Nasir Shaikh
31093,Kannan


Question 9 :
Find all the actors that made more movies with Yash Chopra than any other.

In [11]:
query9 = """  
SELECT p.Name, list.Count FROM Person p JOIN 
(SELECT mc.PID,COUNT(mc.PID) as Count FROM M_Cast mc WHERE TRIM(MID) In
(SELECT md.MID FROM M_Director md WHERE TRIM(PID) IN
(SELECT p.PID FROM Person p WHERE TRIM(p.NAME) = 'Yash Chopra')) GROUP BY mc.PID )as list 
ON TRIM(p.PID) = TRIM(list.PID) 
ORDER BY list.count DESC
"""
ques9 = pd.read_sql_query(query9,conn)
ques9

,Name,Count
0,Jagdish Raj,11
1,Manmohan Krishna,10
2,Iftekhar,9
3,Vikas Anand,8
4,Madan Puri,8
...,...,...
425,Ramchandra,1
426,Sandow S. Sethi,1
427,Naval,1
428,Prem Sood,1


Question 10 :
The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [12]:
query10 = """
SELECT  TRIM(p.NAME) as Actor_Name FROM Person p WHERE TRIM(PID) IN
(SELECT TRIM(mc.PID) as S2ID FROM M_Cast mc WHERE TRIM(MID) IN
(SELECT TRIM(mc.MID) FROM M_cast mc WHERE TRIM(PID) IN
(SELECT TRIM(mc.PID) FROM M_Cast mc WHERE TRIM(MID) IN
(SELECT TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
(SELECT TRIM(p.PID) FROM Person p WHERE TRIM(p.NAME) ='Shah Rukh Khan'))))
EXCEPT
SELECT TRIM(mc.PID) FROM M_Cast mc WHERE TRIM(mc.MID) IN
(SELECT TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
(SELECT TRIM(p.PID) FROM Person p WHERE TRIM(p.NAME) ='Shah Rukh Khan')))
"""

ques10 = pd.read_sql_query(query10,conn)
ques10

,Actor_Name
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long
...,...
25693,Minoo Mehtab
25694,Hayley Cleghorn
25695,Nirvasha Jithoo
25696,Kamal Maharshi
